In [3]:
import icepyx as ipx
import os
from pprint import pprint
from PackageDeepLearn.utils import file_search_wash as fsw

# 数据索引与下载

## 数据基本参数定义与输出

In [6]:
# %%
beam_list = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
var_list = ['h_ph', 'lat_ph', 'lon_ph', 'quality_ph', 'signal_conf_ph']
START_DATE = '2023-01-01'
END_DATE = '2023-12-31'
short_name = 'ATL03' # ATL06、ATL08
spatial_extent = r'D:\BaiduSyncdisk\03_数据与总结\边界信息\SETP_Boundary\SETP_Boundary.shp'
# spatial_extent = [(-55, 68), (-55, 71), (-48, 71), (-48, 68), (-55, 68)]
date_range = {"start_date": START_DATE, "end_date": END_DATE}

# %%
# Specify the version or use the latest version
region_a = ipx.Query(short_name, spatial_extent, date_range, version='006')
region_a.avail_granules()



{'Number of available granules': 222,
 'Average size of granules (MB)': 4312.988787169929,
 'Total size of all granules (MB)': 957483.5107517242}

In [7]:
region_a.order_vars.append(var_list=['dist_ph_along'])

In [8]:
region_a.order_vars.wanted

{'dist_ph_along': ['gt1l/heights/dist_ph_along',
  'gt1r/heights/dist_ph_along',
  'gt2l/heights/dist_ph_along',
  'gt2r/heights/dist_ph_along',
  'gt3l/heights/dist_ph_along',
  'gt3r/heights/dist_ph_along']}

In [ ]:
# %%
path = r'D:\Dataset_and_Demo\2023_ATL03'
region_a.download_granules(path) 

In [ ]:
# 设置子集参数
# region_a.order_vars.remove(all=True)
# region_a.order_vars.append(beam_list=beam_list, var_list=var_list)
# region_a.subsetparams(Coverage=region_a.order_vars.wanted)

## 数据下载

In [ ]:
path = r'D:\Dataset_and_Demo\2023_ATL03'
region_a.download_granules(path) 

# 数据载入与分析

### 载入单个本地数据

In [ ]:
import os
import glob
import h5py
import pandas as pd
from tqdm import tqdm
import time

In [ ]:
ATL03 = r'D:\Dataset_and_Demo\ATL03\ATL03_20190102184312_00810210_006_02.h5'
ATL06 = r'D:\Dataset_and_Demo\ATL06\ATL06_20190102184312_00810210_006_02.h5'

# 用于将 ICESat-2 数据读取为指定格式的数据对象
reader = ipx.Read(ATL06)

# 返回与正在读入的数据关联的变量对象
# 弱光束:gt1l、gt2l、gt3l ; 强束光:gt1r、gt2r、gt3r
reader.vars.append(beam_list=['gt1l', 'gt3r'], var_list=['h_li', "latitude", "longitude"])

ds = reader.load()
print(ds)
# ds.plot.scatter(x="longitude", y="latitude", hue="h_li", vmin=-100, vmax=2000)

### 载入文件夹所有数据
必须是相同类型，如ATL08

In [ ]:
file_list = fsw.search_files(r'D:\Dataset_and_Demo\2023_ATL03','.h5')
reader = ipx.Read(file_list, glob_kwargs={'recursive': True})

In [ ]:
print('数据类型 = {}'.format(reader.product))
print('数据条目 = {}'.format(reader.filelist))
print('数据变量 = {}'.format(reader.vars.parse_var_list(reader.vars.avail())))

数据类型 = ATL03
数据条目 = ['D:\\Dataset_and_Demo\\2023_ATL03\\processed_ATL03_20230102073623_01951802_006_02.h5', 'D:\\Dataset_and_Demo\\2023_ATL03\\processed_ATL03_20230106072759_02561802_006_02.h5', 'D:\\Dataset_and_Demo\\2023_ATL03\\processed_ATL03_20230110071941_03171802_006_02.h5', 'D:\\Dataset_and_Demo\\2023_ATL03\\processed_ATL03_20230114071125_03781802_006_02.h5', 'D:\\Dataset_and_Demo\\2023_ATL03\\processed_ATL03_20230115064547_03931802_006_02.h5', 'D:\\Dataset_and_Demo\\2023_ATL03\\processed_ATL03_20230119063718_04541802_006_02.h5']
数据变量 = ({'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'], 'data_end_utc': ['ancillary_data/data_end_utc'], 'data_start_utc': ['ancillary_data/data_start_utc'], 'end_delta_time': ['ancillary_data/end_delta_time'], 'granule_end_utc': ['ancillary_data/granule_end_utc'], 'granule_start_utc': ['ancillary_data/granule_start_utc'], 'start_delta_time': ['ancillary_data/start_delta_time'], 'delta_time': ['gt1l/geolocation/delta_time', 'gt1l/heights

### 创建独立的变量对象

In [ ]:
v = ipx.Variables(path=ATL06)
# v.avail()
# v = ipx.Variables(product='ATL03', version='006')
# v.avail()

v.parse_var_list(v.avail())

In [ ]:
import xarray as xr

xr.open_dataset(ATL06,group='gt1r/land_ice_segments',engine='h5netcdf')